In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
import sys

sys.path.append("../")

from tokenizers import Tokenizer
import sys

import matplotlib.pyplot as plt
import numpy as np
import collections
import torch

from data import datasets
from training import trainer
from modules import ind_generator
import attr

import plots, utilities

from transformers.tokenization_utils_fast import PreTrainedTokenizerFast
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [52]:
dataset = datasets.CharSMILESChEMBLIndications()

In [53]:
model = ind_generator.SmilesIndGeneratorRNN(
    vocab_size=dataset.vocab_size,
    num_indications=dataset.num_indications,
    num_hiddens=250,
    num_layers=2,
    learning_rate = 0.001,
    weight_decay = 0.01
)

In [54]:
model_trainer = trainer.Trainer(max_epochs=4, init_random=None, clip_grads_norm=1.0)
model_trainer.fit(model, dataset)

Epoch 1/4: Train Loss: 0.4966, Val Loss: 0.3384
Epoch 1/4: Train Loss: 0.4966, Val Loss: 0.3384
Epoch 2/4: Train Loss: 0.3226, Val Loss: 0.2685
Epoch 2/4: Train Loss: 0.3226, Val Loss: 0.2685


KeyboardInterrupt: 

In [ ]:
print(dataset.indications_names)

['mesh_heading_AIDS Dementia Complex', 'mesh_heading_AIDS-Associated Nephropathy', 'mesh_heading_AIDS-Related Complex', 'mesh_heading_Abdominal Abscess', 'mesh_heading_Abdominal Neoplasms', 'mesh_heading_Abdominal Pain', 'mesh_heading_Abortion, Habitual', 'mesh_heading_Abortion, Spontaneous', 'mesh_heading_Abruptio Placentae', 'mesh_heading_Abscess', 'mesh_heading_Acanthamoeba Keratitis', 'mesh_heading_Acid-Base Imbalance', 'mesh_heading_Acidosis', 'mesh_heading_Acidosis, Lactic', 'mesh_heading_Acidosis, Renal Tubular', 'mesh_heading_Acne Vulgaris', 'mesh_heading_Acquired Immunodeficiency Syndrome', 'mesh_heading_Acromegaly', 'mesh_heading_Acute Chest Syndrome', 'mesh_heading_Acute Coronary Syndrome', 'mesh_heading_Acute Kidney Injury', 'mesh_heading_Acute Lung Injury', 'mesh_heading_Acute Pain', 'mesh_heading_Addison Disease', 'mesh_heading_Adenocarcinoma', 'mesh_heading_Adenocarcinoma of Lung', 'mesh_heading_Adenocarcinoma, Bronchiolo-Alveolar', 'mesh_heading_Adenocarcinoma, Mucinous

In [49]:
def generate_sequence(prefix, indications_tensor, num_chars, model, char_to_idx_mapping, idx_to_char_mapping, temperature = 0.0, device=None):
    """
    Simple character-by-character generation function.
    """

    def decode_indices_to_string(encoded_indices: list, idx_to_char_mapping: dict[int, str]):
        decoded = ''.join([idx_to_char_mapping[int(inx)] for inx in encoded_indices])
        return decoded

    def encode_string_to_indices(smiles_string: str, char_to_idx_mapping: dict[str, int]):
        encoded = [char_to_idx_mapping[c] for c in smiles_string]
        return encoded

    model.eval()
    generated = prefix
    
    # Ensure device is correct
    state = indications_tensor.unsqueeze(0).to(device)

    with torch.no_grad():
        for i in range(num_chars):
            # Encode current text to indices
            encoded_indices = encode_string_to_indices(generated, char_to_idx_mapping)
            
            # Convert to tensor and create one-hot encoding
            indices_tensor = torch.tensor(encoded_indices, device=device)
            input_tensor = torch.nn.functional.one_hot(indices_tensor, num_classes=len(char_to_idx_mapping)).float()
            
            # Get prediction
            output, state = model(input_tensor.unsqueeze(0), state) 
            
            # Get most likely next token
            if temperature > 0:
                # Apply temperature scaling
                output = output / temperature
                probabilities = torch.softmax(output, dim=-1)
                next_token = torch.multinomial(probabilities[0, -1, :], num_samples=1).item()
            else:
                # Default to argmax if temperature is 0
                next_token = output[0, -1, :].argmax().item()
            
            # Decode and append
            next_char = idx_to_char_mapping[next_token]

            if next_token == dataset.padding_index or next_char == "":
                break

            
            generated += next_char
            
    return generated

In [50]:
generate_sequence(
    prefix="C",
    indications_tensor = dataset.get_indications_tensor(dataset.indications_names[0]),
    num_chars=100,
    model=model,
    char_to_idx_mapping=dataset.char_to_idx,
    idx_to_char_mapping=dataset.idx_to_char,
    temperature=0.0,
    device=model_trainer.device
)

RuntimeError: Number of dimensions of repeat dims can not be smaller than number of dimensions of tensor